In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import lightgbm as lgbm

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from itertools import product

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
  


In [ ]:
dpath  = '../input/competitive-data-science-predict-future-sales/'


In [ ]:
train = pd.read_csv(dpath + 'sales_train.csv')
test = pd.read_csv(dpath + 'test.csv')

shops = pd.read_csv(dpath + 'shops.csv')

items = pd.read_csv(dpath + 'items.csv')
cats = pd.read_csv(dpath + 'item_categories.csv')



In [ ]:
temp2=train.groupby('date')['item_id'].count()
df=pd.DataFrame(temp2)
df.rename(columns={'item_id':'item_id_cnt'},inplace=True)
df.plot()

In [ ]:
temp2=train[train['item_id']==5037][train['shop_id']==5][['date','item_cnt_day']]
#Convert datetime type
import datetime
def str_to_datetime(s):
    split=s.split('.')
    d,m,y=int(split[0]),int(split[1]),int(split[2])
    return datetime.datetime(year=y,month=m,day=d)
temp2['date']=temp2['date'].apply(str_to_datetime).sort_values()
temp2.head()
#Now proper format

In [ ]:
f,ax=plt.subplots(figsize=(8,6))
sns.boxplot(x=train['item_price'])

In [ ]:
train = train[(train.item_price < 250000 )& (train.item_cnt_day < 1000)]
train.loc[train['item_cnt_day'] < 1, "item_cnt_day"] = 0

## SHOPS DATA SET CLEANING

In [ ]:
shops['city']=shops['shop_name'].str.split(" ").map(lambda x:x[0])
shops['category']=shops['shop_name'].str.split(" ").map(lambda x:x[1])
shops.head(3)

In [ ]:
#Encoder only the main categories, which are most repeated. Put OTHERS for the ones that are not too much repeated.
category=[]
for cat in shops['category'].unique():
    if len(shops[shops['category']==cat])>=5:
        category.append(cat)
shops['category']=shops['category'].apply(lambda x: x if (x in category) else 'others')
shops.head(2)

In [ ]:
shops['category']=LabelEncoder().fit_transform(shops.category)
shops['city']=LabelEncoder().fit_transform(shops.city)
shops.head(2)

In [ ]:
shops=shops[['shop_id','city','category']]
shops.head(2)

## CATEGORIES DATA SET CLEANING

In [ ]:
cats['type']=cats['item_category_name'].apply(lambda x: x.split(' ')[0]).astype(str)
cats.head(2)

In [ ]:
#Encoder only the main categories, which are most repeated. Put OTHERS for the ones that are not too much repeated.
categ=[]
for cat in cats.type.unique():
    if len(cats[cats.type==cat])>=5:
        categ.append(cat)
cats.type=cats.type.apply(lambda x: x if (x in categ) else 'others')
cats.head(2)

In [ ]:
cats['type_code']=LabelEncoder().fit_transform(cats['type'])
cats.head(2)

In [ ]:
#Seperating category name into two new columns.
cats['split']=cats['item_category_name'].apply(lambda x: x.split('-'))
cats['sub_type']=cats['split'].apply(lambda x: x[1].strip() if len(x)>1 else x[0].strip())
cats.head(2)

In [ ]:
cats['sub_type_code']=LabelEncoder().fit_transform(cats.sub_type)
cats=cats[['item_category_id','type_code','sub_type_code']]
cats.head(2)

## ITEMS DATA SET CLEANING

In [ ]:
import re
def name_correction(x):
    x = x.lower() # all letters lower case
    x = x.partition('[')[0] # partition by square brackets
    x = x.partition('(')[0] # partition by curly brackets
    x = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', x) # remove special characters
    x = x.replace('  ', ' ') # replace double spaces with single spaces
    x = x.strip() # remove leading and trailing white space
    return x

In [ ]:
# split item names by first bracket
items["name1"], items["name2"] = items['item_name'].str.split("[", 1).str
items["name1"], items["name3"] = items['item_name'].str.split("(", 1).str

# replace special characters and turn to lower case
items["name2"] = items['name2'].str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()
items["name3"] = items['name3'].str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()
items = items.fillna('0')


In [ ]:
#Word correction
items["item_name"] = items["item_name"].apply(lambda x: name_correction(x))
items.head(3)

In [ ]:
#Extract key words of name2 column
items["type"] = items['name2'].apply(lambda x: x[0:8] if x.split(" ")[0] == "xbox" else x.split(" ")[0] )
items.head(3)

In [ ]:
#Cleaning
items.loc[(items.type == "x360") | (items.type == "xbox360") | (items.type == "xbox 360") ,"type"] = "xbox 360"
items.loc[ items.type == "", "type"] = "mac"
items.type = items.type.apply( lambda x: x.replace(" ", "") )
items.loc[ (items.type == 'pc' )| (items.type == 'pс') | (items.type == "pc"), "type" ] = "pc"
items.loc[ items.type == 'рs3' , "type"] = "ps3"

In [ ]:
#"Item_id" counting grouped by "type". The colum "item_id" will be for the new count after grouping
group_sum = items.groupby(["type"]).agg({"item_id": "count"})
group_sum = group_sum.reset_index()
group_sum.head(3)

In [ ]:
group_sum.type.unique()

In [ ]:
#Borrar categorias con pocos items
drop_cols = []
for cat in group_sum.type.unique():
    if group_sum.loc[(group_sum.type == cat), "item_id"].values[0] <40:
        drop_cols.append(cat)
drop_cols

In [ ]:
items.head(3)

In [ ]:
items.name2 = items.name2.apply( lambda x: "other" if (x in drop_cols) else x )
items = items.drop(["type"], axis = 1)

In [ ]:
items[items['name2']=="other"]
#only two item_id had a "name2" column with less than 40 values

In [ ]:
items.name2 = LabelEncoder().fit_transform(items.name2)
items.name3 = LabelEncoder().fit_transform(items.name3)
items.drop(["item_name", "name1"],axis = 1, inplace= True)
train.head(3)

## SALES (Train) DATAFRAME CLEANING

In [ ]:
#Esto es por que se necesita un seguimiento de cada item en cada shop en cada mes
from itertools import product
matrix=[]
cols  = ["date_block_num", "shop_id", "item_id"]
for i in range(34):
    sales=train[train.date_block_num==i]
    matrix.append(np.array(list(product( [i], sales.shop_id.unique(), sales.item_id.unique()))))
    
matrix=pd.DataFrame(np.vstack(matrix),columns=cols)
matrix.head(3)

In [ ]:
matrix['date_block_num']=matrix['date_block_num'].astype(int)
matrix['shop_id']=matrix['shop_id'].astype(np.int8)
matrix['item_id']=matrix['item_id'].astype(np.int16)
matrix.sort_values(cols,inplace=True)
matrix.head(3)

In [ ]:
train['revenue']=train['item_cnt_day']*train['item_price']
train.head(3)

In [ ]:
group=train.groupby(cols).agg({'item_cnt_day':['sum']})
group.columns=['item_cnt_month']
group.reset_index(inplace=True)
group.head(2)

In [ ]:
#Poner Items_cnt_month in the matrix df
matrix=pd.merge(matrix,group,on=cols,how='left')
matrix['item_cnt_month']=matrix['item_cnt_month'].fillna(0).astype(np.float16)
matrix.head(2)

In [ ]:
#Juntar TEST df con el matrix df.
test['date_block_num']=34
test["date_block_num"] = test["date_block_num"].astype(np.int8)
test['shop_id']=test['shop_id'].astype(np.int8)
test['item_id']=test['item_id'].astype(np.int16)
test.head(2)

In [ ]:
test.drop('ID',inplace=True,axis=1)
test.head(2)

In [ ]:
matrix=pd.concat([matrix,test],ignore_index=True,sort=False,keys=cols)
matrix.fillna(0,inplace=True)
matrix.tail(3)

## JUNTANDO SHOPS Df al MATRIX df

In [ ]:
shops.head(2)

In [ ]:
#Two new columns to be added: City and Category
matrix=pd.merge(matrix,shops,on=['shop_id'],how='left')
matrix.head(3)

## JUNTANDO ITEMS Df al MATRIX df

In [ ]:
items.head(2)

In [ ]:
#Four new columns to be added: item_category, name2, name3 and type
matrix=pd.merge(matrix,items,on='item_id',how='left')
matrix.head(3)

## JUNTANDO CATEGORIES Df al MATRIX df

In [ ]:
cats.head(2)

In [ ]:
#TWO new columns to be added: type_code, sub_Type_code
matrix=pd.merge(matrix,cats,on='item_category_id',how='left')
matrix.head(3)

In [ ]:
matrix["city"] = matrix["city"].astype(np.int8)
matrix["category"] = matrix["category"].astype(np.int8)
matrix["item_category_id"] = matrix["item_category_id"].astype(np.int8)
matrix["sub_type_code"] = matrix["sub_type_code"].astype(np.int8)
matrix["name2"] = matrix["name2"].astype(np.int8)
matrix["name3"] = matrix["name3"].astype(np.int16)
matrix["type_code"] = matrix["type_code"].astype(np.int8)

## CREATING LAGS TO THE FEATURES

In [ ]:
def lag_feature(df,lags,cols ):
    for col in cols:
        print('Adding lag feature in ',col)
        tmp=df[['date_block_num','shop_id','item_id',col]]
        for i in lags:
            shifted=tmp.copy()
            shifted.columns=['date_block_num','shop_id','item_id',col+'_shifted_'+str(i)]
            shifted.date_block_num=shifted.date_block_num+i
            df=pd.merge(df,shifted,on=['date_block_num','shop_id','item_id'],how='left')
    return df

#### LAG FOR ITEM_CNT_MONTH

In [ ]:
matrix=lag_feature(matrix,[1,2,3],["item_cnt_month"])
matrix.head(3)

## AVERAGES COUPLES TO DO:

### FOR ITEM_COUNT


**DATE_BLOCK_NUM  &  ITEM_CATEGORY_ID  (Lag)**

**DATE_BLOCK_NUM  &  CATEGORY (Lag)**
 
**DATE_BLOCK_NUM (Lag)**

**DATE_BLOCK_NUM  &  ITEM_ID (Lag)**



### FOR ITEM PRICE

**ITEM_ID**

**DATE_BLOCK_NUM  &  ITEM_ID (Lag)**

#### LAG FOR DATE_ITEM_CAT_AVG

In [ ]:
#Averaging item_cnt_month by "date_block_num and item_category_id"
group=matrix.groupby(['date_block_num','item_category_id']).agg({'item_cnt_month':['mean']})
group.columns=['date_item_cat_avg']
group.reset_index(inplace=True)
group.head(3)

In [ ]:
matrix=pd.merge(matrix,group,on=['date_block_num','item_category_id'],how='left')
matrix['date_item_cat_avg']=matrix['date_item_cat_avg'].astype(np.float16)
matrix.head(3)

In [ ]:
matrix=lag_feature(matrix,[1,2],['date_item_cat_avg'])
matrix.drop(['date_item_cat_avg'],axis=1,inplace=True)
matrix.head(3)

#### LAG FOR DATA_CAT_AVG

In [ ]:
#Averaging item_cnt_month by "date_block_num and category"
group=matrix.groupby(['date_block_num','category']).agg({'item_cnt_month':['mean']})
group.columns=['date_cat_avg']
group.reset_index(inplace=True)
group.head(2)

In [ ]:
matrix=pd.merge(matrix,group,on=['date_block_num','category'],how='left')
matrix['date_cat_avg']=matrix['date_cat_avg'].astype(np.float16)
matrix.head(3)

In [ ]:
matrix=lag_feature(matrix,[1,2],['date_cat_avg'])
matrix.drop(['date_cat_avg'],axis=1,inplace=True)
matrix.head(3)

#### LAG FOR DATE_AVG_ITEM_CNT

In [ ]:
group=matrix.groupby(['date_block_num']).agg({'item_cnt_month':['mean']})
group.columns=['date_avg_item_cnt']
group.reset_index(inplace=True)
group.head(3)

In [ ]:
matrix=pd.merge(matrix,group,on='date_block_num',how='left')
matrix.date_avg_item_cnt = matrix["date_avg_item_cnt"].astype(np.float16)
matrix.head(3)

In [ ]:
matrix=lag_feature(matrix,[1,2],["date_avg_item_cnt"])
matrix.drop(['date_avg_item_cnt'],inplace=True,axis=1)
matrix.head(3)

#### LAG FOR DATE_ITEM_AVG_ITEM_CNT

In [ ]:
group=matrix.groupby(['date_block_num','item_id']).agg({'item_cnt_month':['mean']})
group.columns=['date_item_avg_item_cnt']
group.reset_index(inplace=True)
group.head(3)

In [ ]:
matrix=pd.merge(matrix,group,on=['date_block_num','item_id'],how='left')
matrix.date_item_avg_item_cnt=matrix['date_item_avg_item_cnt'].astype(np.float16)
matrix.head(3)

In [ ]:
matrix=lag_feature(matrix,[1,2,3],['date_item_avg_item_cnt'])
matrix.drop(['date_item_avg_item_cnt'],inplace=True,axis=1)
matrix.head(3)

#### LAG FOR DATE_ITEM_ID_PRICE_AVG

In [ ]:
group=train.groupby(['item_id']).agg({'item_price':['mean']})
group.columns=['item_id_price_avg']
group.reset_index(inplace=True)
group.head(3)

In [ ]:
matrix=pd.merge(matrix,group,on=['item_id'],how='left')
matrix['item_id_price_avg']=matrix['item_id_price_avg'].astype(np.float16)
matrix.head(3)      


In [ ]:
group=train.groupby(['date_block_num','item_id']).agg({'item_price':['mean']})
group.columns=['date_item_id_price_avg']
group.reset_index(inplace=True)
group.head(3)

In [ ]:
matrix=pd.merge(matrix,group,on=['date_block_num','item_id'],how='left')
matrix['date_item_id_price_avg']=matrix['date_item_id_price_avg'].astype(np.float16)
matrix.head(3)

In [ ]:
matrix=lag_feature(matrix,[1,2,3],['date_item_id_price_avg'])
matrix.head(3)

### DROPPING FEATURES

In [ ]:
for i in [1,2,3]:
    matrix['delta_price_shifted_'+str(i)]=(matrix['date_item_id_price_avg_shifted_'+str(i)]-matrix['item_id_price_avg'])/matrix['item_id_price_avg']


features_to_drop = ["item_id_price_avg", "date_item_id_price_avg"]

matrix.drop(features_to_drop, axis = 1, inplace = True)

In [ ]:
matrix.info()

### FORECASTING MODEL LGBM

In [ ]:
X_train = matrix[matrix.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = matrix[matrix.date_block_num < 33]['item_cnt_month']
X_valid = matrix[matrix.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = matrix[matrix.date_block_num == 33]['item_cnt_month']
X_test = matrix[matrix.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [ ]:
Y_train = Y_train.clip(0, 20)
Y_valid = Y_valid.clip(0, 20)

In [ ]:
train_data = lgbm.Dataset(data=X_train, label=Y_train)
valid_data = lgbm.Dataset(data=X_valid, label=Y_valid)

In [ ]:
# configuracion de los parametros
params = {"objective" : "regression", 
          "metric" : "rmse", 
          'n_estimators':10000, 
          'early_stopping_rounds':50,
          "num_leaves" : 2**7-1, 
          "learning_rate" : 0.01, 
          "bagging_fraction" : 0.9,
          "feature_fraction" : 0.3, 
          "bagging_seed" : 0
         }
         
          
# entrenamiento
lgbm_model = lgbm.train(params, 
                        train_data, 
                        valid_sets=[train_data, valid_data], 
                        verbose_eval=1000) 

In [ ]:
lgbm.plot_importance(
    lgbm_model, 
    max_num_features=50, 
    importance_type='gain', 
    figsize=(12,8));

In [ ]:
pred=lgbm_model.predict(X_test)

In [ ]:
len(pred)

In [ ]:
output = pd.DataFrame({'ID': test.index, 'item_cnt_month': pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")